In [6]:
import requests
import json

In [ ]:
# 利用 requests 對 API 來源發送一個請求
url = 'https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json'
res = requests.get(url)

# 將請求回應的內容存成一個字串格式
text = res.text

# json格式的字串解析成字典或列表
data = json.loads(text)

print(data)

In [52]:
# 計算各縣市藥局數量
print(len(data['features']))
data['features'][0].keys() # ['type', 'properties', 'geometry']
data['features'][0]['properties'] # 基本資料在properties裡

length = len(data['features'])
med_count = {}
for i in range(length):
    county = data['features'][i]['properties']['county']
    if county == '':
        county = '未知'
    if county not in med_count:
        med_count[county] = 0
    med_count[county] += 1

med_count_sort = dict(sorted(med_count.items(), key=lambda x: x[1], reverse=True)) # return dict
print(med_count_sort)

3293
{'新北市': 501, '高雄市': 422, '臺中市': 420, '臺北市': 339, '臺南市': 271, '桃園市': 264, '彰化縣': 179, '屏東縣': 140, '雲林縣': 129, '嘉義縣': 84, '宜蘭縣': 76, '嘉義市': 67, '南投縣': 67, '基隆市': 57, '苗栗縣': 56, '未知': 49, '新竹縣': 46, '花蓮縣': 46, '新竹市': 39, '臺東縣': 23, '澎湖縣': 11, '金門縣': 6, '連江縣': 1}


In [53]:
# 計算各縣市的成人剩餘口罩數量
adult_mask_count = {}

for i in range(length):
    county = data['features'][i]['properties']['county']
    if county == '':
        county = '未知'
    adult_mask = data['features'][i]['properties']['mask_adult']
    if county not in adult_mask_count:
        adult_mask_count[county] = 0
    adult_mask_count[county] += adult_mask

adult_mask_count_sort = dict(sorted(adult_mask_count.items(), key=lambda x: x[1], reverse=True))
print(adult_mask_count_sort)

{'新北市': 670270, '臺中市': 536710, '高雄市': 424170, '臺南市': 393900, '桃園市': 331940, '臺北市': 322490, '彰化縣': 219400, '雲林縣': 196990, '屏東縣': 184250, '苗栗縣': 131080, '嘉義縣': 113180, '嘉義市': 107840, '宜蘭縣': 106900, '南投縣': 91240, '基隆市': 80100, '未知': 69210, '花蓮縣': 57330, '新竹縣': 51350, '新竹市': 45630, '臺東縣': 28510, '金門縣': 15510, '澎湖縣': 12600, '連江縣': 7930}


In [55]:
# 計算各縣市的小孩剩餘口罩數量
child_mask_count = {}

for i in range(length):
    county = data['features'][i]['properties']['county']
    if county == '':
        county = '未知'
    child_mask = data['features'][i]['properties']['mask_child']
    if county not in child_mask_count:
        child_mask_count[county] = 0
    child_mask_count[county] += child_mask

child_mask_count_sort = dict(sorted(child_mask_count.items(), key=lambda x: x[1], reverse=True))
print(child_mask_count_sort)

{'新北市': 539260, '臺中市': 470910, '高雄市': 421180, '臺北市': 344150, '臺南市': 327950, '桃園市': 270810, '彰化縣': 220060, '屏東縣': 169820, '雲林縣': 165570, '嘉義縣': 112370, '嘉義市': 97820, '宜蘭縣': 93670, '南投縣': 91630, '基隆市': 84380, '苗栗縣': 82630, '花蓮縣': 57190, '未知': 56130, '新竹縣': 55630, '新竹市': 43580, '臺東縣': 25810, '澎湖縣': 16510, '金門縣': 14980, '連江縣': 3470}
